<center>

# $\textbf{Processing All in One}$

<center>

### $\textbf{Code}$

In [1]:
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import explode, col, lit, array, struct, regexp_replace, year, sum, col, when

In [2]:
inicio = time.time()

In [3]:
spark = SparkSession.builder.appName('AllInOne').master("local").enableHiveSupport().getOrCreate()
spark

### $\textbf{Carregar os Datasets}$

In [4]:
# Creating dataframes from the csv's and xlsx files and infering the schema
df_covid = spark.read.load("Files/Covid.csv", format="csv", sep=",", inferschema="true", header="true")
df_gdp = spark.read.load("Files/GDP.csv", format="csv", sep=",", inferschema="true", header="true")
df_migration = spark.read.load("Files/Migration.csv", format="csv", sep=",", inferschema="true", header="true")
df_population = spark.read.load("Files/Population.csv", format="csv", sep=",", inferschema="true", header="true")
df_tax = spark.read.load("Files/Tax.csv", format="csv", sep=",", inferschema="true", header="true")
df_unemployment = spark.read.load("Files/Unemployment.csv", format="csv", sep=",", inferschema="true", header="true")

In [5]:
# Step 4: Read the Excel file using pandas
pandas_df = pd.read_excel("Files/Inflation.xlsx")

# Replace "no data" with NaN
pandas_df.replace("no data", float("nan"), inplace=True)

# Extract column names from the first row
column_names = [str(col) for col in pandas_df.columns]

# Step 5: Define the schema for the Spark DataFrame
schema_fields = [StructField(column_names[0], StringType(), True)] + \
                [StructField(col, DoubleType(), True) for col in column_names[1:]]

# Create schema
schema = StructType(schema_fields)

# Step 6: Convert the pandas DataFrame to a Spark DataFrame with the specified schema
df_inflation = spark.createDataFrame(pandas_df, schema=schema)

### $\textbf{Covid - Processamento}$

In [6]:
# Select the columns you're interested in
df_covid = df_covid.select("location","date","total_cases")

# Rename the columns "location" and "total_cases"
df_covid = df_covid.withColumnRenamed("location","country")
df_covid = df_covid.withColumnRenamed("total_cases","covid_cases")

# Extract year from 'day' column
df_covid = df_covid.withColumn("year", year("date"))
df_covid = df_covid.drop("date")

# Replace null values in 'total_cases' column with 0
df_covid = df_covid.withColumn("covid_cases", when(df_covid["covid_cases"].isNull(), 0).otherwise(df_covid["covid_cases"]))

# Remove non-numeric characters from 'total_cases' column
df_covid = df_covid.withColumn("covid_cases", regexp_replace("covid_cases", "[^0-9]", ""))

# Cast columns to their desired types
df_covid = df_covid.withColumn("country", col("country").cast("string"))
df_covid = df_covid.withColumn("year", col("year").cast("int"))
df_covid = df_covid.withColumn("covid_cases", col("covid_cases").cast("int"))

# Filter out any null values after cleaning
df_covid = df_covid.filter(col("covid_cases").isNotNull())

# Calculate total cases per year
df_covid = df_covid.groupBy("country", "year").agg(sum("covid_cases"))

# Order by country and then by year
df_covid = df_covid.orderBy("country", "year")

# Rename 'sum(covid_cases)' to 'covid_cases'
df_covid = df_covid.withColumnRenamed("sum(covid_cases)","covid_cases")

# Rename the column
df_covid = df_covid.withColumnRenamed("Micronesia (country)","Micronesia")
df_covid = df_covid.withColumnRenamed("Saint Martin (French part)","Saint Martin")
df_covid = df_covid.withColumnRenamed("Sint Maarten (Dutch part)","Sint Maarten")
df_covid = df_covid.withColumnRenamed("Brunei","Brunei Darussalam")
df_covid = df_covid.withColumnRenamed("Cape Verde","Cabo Verde")
df_covid = df_covid.withColumnRenamed("Laos","Lao PDR")
df_covid = df_covid.withColumnRenamed("United States Virgin Islands","Virgin Islands (U.S.)")
df_covid = df_covid.withColumnRenamed("Turkey","Turkiye")
df_covid = df_covid.withColumnRenamed("Syria","Syrian Arab Republic")
df_covid = df_covid.withColumnRenamed("Timor","Timor-Leste")
df_covid = df_covid.withColumnRenamed("Russia","Russian Federation")
df_covid = df_covid.withColumnRenamed("Congo","Democratic Republic of the Congo")
df_covid = df_covid.withColumnRenamed("Slovakia","Slovenia")

# Drop the specified lines
countries_to_remove = ["Africa", "Europe", "Europe Union", "Asia", "Lower middle income", "Upper middle income", "World"]
df_covid = df_covid.filter(~df_covid["country"].isin(countries_to_remove))

### $\textbf{GDP - Processamento}$

In [7]:
# Assuming df_gdp is properly defined DataFrame
df_gdp = df_gdp.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("gdp")) 
    for year in range(2000, 2025)
])).alias("data")).selectExpr("country", "data.year", "data.gdp")

# Cast columns to their desired types
df_gdp = df_gdp.withColumn("country", col("country").cast("string"))
df_gdp = df_gdp.withColumn("year", col("year").cast("int"))
df_gdp = df_gdp.withColumn("gdp", col("gdp").cast("double"))

# Filter data for years greater than 2010 and lower than 2024
df_gdp = df_gdp.filter(df_gdp["year"] > 2010)
df_gdp = df_gdp.filter(df_gdp["year"] < 2024)

# Order by country and then by year
df_gdp = df_gdp.orderBy("country", "year")

# Rename the column
df_gdp = df_gdp.withColumnRenamed("China, People\'s Republic of","China")
df_gdp = df_gdp.withColumnRenamed("Gambia, The","Gambia")
df_gdp = df_gdp.withColumnRenamed("Micronesia, Fed. States of","Micronesia")
df_gdp = df_gdp.withColumnRenamed("South Sudan, Republic of","South Sudan")
df_gdp = df_gdp.withColumnRenamed("Taiwan Province of China","Taiwan")
df_gdp = df_gdp.withColumnRenamed("Türkiye, Republic of","Turkiye")
df_gdp = df_gdp.withColumnRenamed("Bahamas, The","Bahamas")
df_gdp = df_gdp.withColumnRenamed("Syria","Syrian Arab Republic")
df_gdp = df_gdp.withColumnRenamed("Czech Republic","Czechia")
df_gdp = df_gdp.withColumnRenamed("Korea, Republic of","South Korea")
df_gdp = df_gdp.withColumnRenamed("Côte d\'Ivoire","Cote d\'Ivoire")
df_gdp = df_gdp.withColumnRenamed("Hong Kong SAR","Hong Kong")
df_gdp = df_gdp.withColumnRenamed("Lao P.D.R.","Lao PDR")
df_gdp = df_gdp.withColumnRenamed("Congo, Republic of ","Republic of the Congo")
df_gdp = df_gdp.withColumnRenamed("Congo, Dem. Rep. of the","Democratic Republic of the Congo")
df_gdp = df_gdp.withColumnRenamed("Pacific Islands ","Pacific island small states")
df_gdp = df_gdp.withColumnRenamed("North Macedonia ","North Macedonia")

# Columns to drop
countries_to_remove = ["ASEAN-5","Advanced economies", "Africa (Region)","Asia and Pacific", "Australia and New Zealand","Central America",
        "Central Asia and the Caucasus","©IMF, 2023","East Asia","Eastern Europe ","Emerging and Developing Asia",
        "Emerging and Developing Europe","Emerging market and developing economies","Euro area","Europe","European Union",
        "Latin America and the Caribbean","Macao SAR","Major advanced economies (G7)","Middle East (Region)","Middle East and Central Asia",
        "North Africa","North America","North Macedonia","Other advanced economies","South America","South Asia","Southeast Asia",
        "Sub-Saharan Africa","Sub-Saharan Africa (Region)","Western Europe","Western Hemisphere (Region)","World"]

df_gdp = df_gdp.filter(~df_gdp["country"].isin(countries_to_remove))

### $\textbf{Inflation - Processamento}$

In [8]:
# Rename the columns "location" and "total_cases"
df_inflation = df_inflation.withColumnRenamed("Inflation rate, average consumer prices (Annual percent change)","country")

# Assuming df_inflation is properly defined DataFrame
df_inflation = df_inflation.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("inflation")) 
    for year in range(2000, 2025)
])).alias("data")).selectExpr("country", "data.year", "data.inflation")

# Cast columns to their desired types
df_inflation = df_inflation.withColumn("country", col("country").cast("string"))
df_inflation = df_inflation.withColumn("year", col("year").cast("int"))
df_inflation = df_inflation.withColumn("inflation", col("inflation").cast("double"))

# Filter data for years greater than 2010 and lower than 2024
df_inflation = df_inflation.filter(df_inflation["year"] > 2010)
df_inflation = df_inflation.filter(df_inflation["year"] < 2024)

# Order by country and then by year
df_inflation = df_inflation.orderBy("country", "year")

replacements = {
    "China, People's Republic of": 'China',
    'Gambia, The': 'Gambia',
    'Micronesia, Fed. States of': 'Micronesia',
    'South Sudan, Republic of': 'South Sudan',
    'Taiwan Province of China': 'Taiwan',
    'Türkiye, Republic of': 'Turkiye',
    'Bahamas, The': 'Bahamas',
    'Czech Republic': 'Czechia',
    'Syria': 'Syrian Arab Republic',
    "Côte d'Ivoire": "Cote d'Ivoire",
    'Hong Kong SAR': 'Hong Kong',
    'Lao P.D.R.': 'Lao PDR',
    'Korea, Republic of': 'South Korea',
    'Congo, Dem. Rep. of the': 'Democratic Republic of the Congo',
    'Congo, Republic of ': 'Republic of the Congo',
    'Pacific Islands ': 'Pacific island small states',
    'North Macedonia ': 'North Macedonia'
}

# Rename columns based on replacements dictionary
for old_value, new_value in replacements.items():
    df_inflation = df_inflation.withColumnRenamed(old_value, new_value)

# Columns to drop
countries_to_remove = ['ASEAN-5', 
                       'Advanced economies', 
                       'Africa (Region)', 
                       'Asia and Pacific', 
                       'Australia and New Zealand', 
                       'Central America', 
                       'Central Asia and the Caucasus', 
                       '©IMF, 2023',
                       'East Asia',
                       'Eastern Europe ',
                       'Emerging and Developing Asia',
                       'Emerging and Developing Europe',
                       'Emerging market and developing economies',
                       'Euro area',
                       'Europe',
                       'European Union',
                       'Latin America and the Caribbean',
                       'Macao SAR',
                       'Major advanced economies (G7)',
                       'Middle East (Region)',
                       'Middle East and Central Asia',
                       'North Africa',
                       'North America',
                       'North Macedonia',
                       'Other advanced economies',
                       'South America',
                       'South Asia',
                       'Southeast Asia',
                       'Sub-Saharan Africa',
                       'Sub-Saharan Africa (Region)',
                       'Western Europe',
                       'Western Hemisphere (Region)',
                       'World',
                       'nan']

# Drop the specified rows
df_inflation = df_inflation.filter(~df_inflation['country'].isin(countries_to_remove))

# Remove rows with missing values in the 'country' column
df_inflation = df_inflation.dropna(subset=['country'])

### $\textbf{Migration - Processamento}$

In [9]:
# Drop the columns you don't want
df_migration = df_migration.drop("Series Name", "Series Code", "Country Code")

# Rename the column
df_migration = df_migration.withColumnRenamed("Country Name","country")
df_migration = df_migration.withColumnRenamed("2000 [YR2000]","2000")
df_migration = df_migration.withColumnRenamed("2001 [YR2001]","2001")
df_migration = df_migration.withColumnRenamed("2002 [YR2002]","2002")
df_migration = df_migration.withColumnRenamed("2003 [YR2003]","2003")
df_migration = df_migration.withColumnRenamed("2004 [YR2004]","2004")
df_migration = df_migration.withColumnRenamed("2008 [YR2008]","2008")
df_migration = df_migration.withColumnRenamed("2009 [YR2009]","2009")
df_migration = df_migration.withColumnRenamed("2010 [YR2010]","2010")
df_migration = df_migration.withColumnRenamed("2011 [YR2011]","2011")
df_migration = df_migration.withColumnRenamed("2012 [YR2012]","2012")
df_migration = df_migration.withColumnRenamed("2013 [YR2013]","2013")
df_migration = df_migration.withColumnRenamed("2014 [YR2014]","2014")
df_migration = df_migration.withColumnRenamed("2015 [YR2015]","2015")
df_migration = df_migration.withColumnRenamed("2016 [YR2016]","2016")
df_migration = df_migration.withColumnRenamed("2017 [YR2017]","2017")
df_migration = df_migration.withColumnRenamed("2018 [YR2018]","2018")
df_migration = df_migration.withColumnRenamed("2019 [YR2019]","2019")
df_migration = df_migration.withColumnRenamed("2020 [YR2020]","2020")
df_migration = df_migration.withColumnRenamed("2021 [YR2021]","2021")
df_migration = df_migration.withColumnRenamed("2022 [YR2022]","2022")
df_migration = df_migration.withColumnRenamed("2023 [YR2023]","2023")
df_migration = df_migration.withColumnRenamed("2024 [YR2024]","2024")
df_migration = df_migration.withColumnRenamed("2025 [YR2025]","2025")

# Assuming df_migration is properly defined DataFrame
df_migration = df_migration.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("migration")) 
    for year in list(range(2000, 2005)) + list(range(2008, 2025))
])).alias("data")).selectExpr("country", "data.year", "data.migration")

# Cast columns to their desired types
df_migration = df_migration.withColumn("country", col("country").cast("string"))
df_migration = df_migration.withColumn("year", col("year").cast("int"))
df_migration = df_migration.withColumn("migration", col("migration").cast("int"))

# Filter data for years greater than 2010 and lower than 2024
df_migration = df_migration.filter(df_migration["year"] > 2010)
df_migration = df_migration.filter(df_migration["year"] < 2024)

# Order by country and then by year
df_migration = df_migration.orderBy("country", "year")

replacements = {
    "Bahamas, The": "Bahamas",
    "Egypt, Arab Rep.": "Egypt",
    "Micronesia, Fed. Sts.": "Micronesia",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "St. Martin",
    "Venezuela, RB": "Venezuela",
    "Yemen, Rep.": "Yemen",
    "Caribbean small states": "Caribbean",
    "Gambia, The": "Gambia",
    "Hong Kong SAR, China": "Hong Kong",
    "Iran, Islamic Rep.": "Iran",
    "Macao SAR, China": "Macao SAR",
    "Korea, Rep.": "South Korea",
    "Korea, Dem. People's Rep.": "Korea",
    "Congo, Rep.": "Republic of the Congo",
    "Congo, Dem. Rep.": "Democratic Republic of the Congo",
    "Viet Nam": "Vietnam"
}

# Rename columns based on replacements dictionary
for old_value, new_value in replacements.items():
    df_migration = df_migration.withColumnRenamed(old_value, new_value)

# Columns to drop
countries_to_remove = ['Africa Eastern and Southern', 
                       'Africa Western and Central', 
                       'Central Europe and the Baltics', 
                       'Early-demographic dividend', 
                       'East Asia & Pacific', 
                       'East Asia & Pacific (IDA & IBRD countries)', 
                       'East Asia & Pacific (excluding high income)', 
                       'Euro area',
                       'Europe & Central Asia',
                       'Europe & Central Asia (IDA & IBRD countries)',
                       'Europe & Central Asia (excluding high income)',
                       'European Union',
                       'Fragile and conflict affected situations',
                       'Heavily indebted poor countries (HIPC)',
                       'High income',
                       'IBRD only',
                       'IDA & IBRD total',
                       'IDA blend',
                       'IDA only',
                       'IDA total',
                       'Latin America & the Caribbean (IDA & IBRD countries)',
                       'Latin America & Caribbean (excluding high income)',
                       'Least developed countries: UN classification',
                       'Low & middle income',
                       'Low income',
                       'Lower middle income',
                       'Middle East & North Africa',
                       'Middle East & North Africa (IDA & IBRD countries)',
                       'Middle East & North Africa (excluding high income)',
                       'Middle income',
                       'Not classified',
                       'OECD members',
                       'Other small states',
                       'Pacific island small states'
                       'Post-demographic dividend',
                       'Pre-demographic dividend',
                       'Small states',
                       'South Asia',
                       'South Asia (IDA & IBRD)',
                       'Sub-Saharan Africa (IDA & IBRD countries)',
                       'Sub-Saharan Africa (excluding high income)',
                       'Upper middle income',
                       'World'
                       ]

# Drop the specified rows
df_migration = df_migration.filter(~df_migration['country'].isin(countries_to_remove))

# Remove rows with missing values in the 'country' column
df_migration = df_migration.dropna(subset=['country'])

### $\textbf{Population - Processamento}$

In [10]:
# Drop the columns you don't want
df_population = df_population.drop("Series Name", "Series Code", "Country Code")

# Rename the column
df_population = df_population.withColumnRenamed("Country Name","country")
df_population = df_population.withColumnRenamed("2000 [YR2000]","2000")
df_population = df_population.withColumnRenamed("2001 [YR2001]","2001")
df_population = df_population.withColumnRenamed("2002 [YR2002]","2002")
df_population = df_population.withColumnRenamed("2003 [YR2003]","2003")
df_population = df_population.withColumnRenamed("2004 [YR2004]","2004")
df_population = df_population.withColumnRenamed("2005 [YR2005]","2005")
df_population = df_population.withColumnRenamed("2006 [YR2006]","2006")
df_population = df_population.withColumnRenamed("2007 [YR2007]","2007")
df_population = df_population.withColumnRenamed("2008 [YR2008]","2008")
df_population = df_population.withColumnRenamed("2009 [YR2009]","2009")
df_population = df_population.withColumnRenamed("2010 [YR2010]","2010")
df_population = df_population.withColumnRenamed("2011 [YR2011]","2011")
df_population = df_population.withColumnRenamed("2012 [YR2012]","2012")
df_population = df_population.withColumnRenamed("2013 [YR2013]","2013")
df_population = df_population.withColumnRenamed("2014 [YR2014]","2014")
df_population = df_population.withColumnRenamed("2015 [YR2015]","2015")
df_population = df_population.withColumnRenamed("2016 [YR2016]","2016")
df_population = df_population.withColumnRenamed("2017 [YR2017]","2017")
df_population = df_population.withColumnRenamed("2018 [YR2018]","2018")
df_population = df_population.withColumnRenamed("2019 [YR2019]","2019")
df_population = df_population.withColumnRenamed("2020 [YR2020]","2020")
df_population = df_population.withColumnRenamed("2021 [YR2021]","2021")
df_population = df_population.withColumnRenamed("2022 [YR2022]","2022")
df_population = df_population.withColumnRenamed("2023 [YR2023]","2023")
df_population = df_population.withColumnRenamed("2024 [YR2024]","2024")
df_population = df_population.withColumnRenamed("2025 [YR2025]","2025")

# Assuming df_population is properly defined DataFrame
df_population = df_population.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("population")) 
    for year in range(2000, 2025)
])).alias("data")).selectExpr("country", "data.year", "data.population")

# Cast columns to their desired types
df_population = df_population.withColumn("country", col("country").cast("string"))
df_population = df_population.withColumn("year", col("year").cast("int"))
df_population = df_population.withColumn("population", col("population").cast("int"))

# Filter data for years greater than 2010 and lower than 2024
df_population = df_population.filter(df_population["year"] > 2010)
df_population = df_population.filter(df_population["year"] < 2024)

# Order by country and then by year
df_population = df_population.orderBy("country", "year")

replacements = {
    "Bahamas, The": "Bahamas",
    "Egypt, Arab Rep.": "Egypt",
    "Micronesia, Fed. Sts.": "Micronesia",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "St. Martin",
    "Venezuela, RB": "Venezuela",
    "Yemen, Rep.": "Yemen",
    "Caribbean small states": "Caribbean",
    "Gambia, The": "Gambia",
    "Hong Kong SAR, China": "Hong Kong",
    "Iran, Islamic Rep.": "Iran",
    "Congo, Rep.": "Republic of the Congo",
    "Macao SAR, China": "Macao SAR",
    "Korea, Rep.": "South Korea",
    "Korea, Dem. People's Rep.": "Korea",
    "Congo, Dem. Rep.": "Democratic Republic of the Congo",
    "Viet Nam": "Vietnam"
}

# Rename columns based on replacements dictionary
for old_value, new_value in replacements.items():
    df_population = df_population.withColumnRenamed(old_value, new_value)

# Columns to drop
countries_to_remove = ['Africa Eastern and Southern', 
                       'Africa Western and Central', 
                       'Central Europe and the Baltics', 
                       'Early-demographic dividend', 
                       'East Asia & Pacific', 
                       'East Asia & Pacific (IDA & IBRD countries)', 
                       'East Asia & Pacific (excluding high income)', 
                       'Euro area',
                       'Europe & Central Asia',
                       'Europe & Central Asia (IDA & IBRD countries)',
                       'Europe & Central Asia (excluding high income)',
                       'European Union',
                       'Fragile and conflict affected situations',
                       'Heavily indebted poor countries (HIPC)',
                       'High income',
                       'IBRD only',
                       'IDA & IBRD total',
                       'IDA blend',
                       'IDA only',
                       'IDA total',
                       'Latin America & the Caribbean (IDA & IBRD countries)',
                       'Latin America & Caribbean (excluding high income)',
                       'Least developed countries: UN classification',
                       'Low & middle income',
                       'Low income',
                       'Lower middle income',
                       'Middle East & North Africa',
                       'Middle East & North Africa (IDA & IBRD countries)',
                       'Middle East & North Africa (excluding high income)',
                       'Middle income',
                       'Not classified',
                       'OECD members',
                       'Other small states',
                       'Pacific island small states'
                       'Post-demographic dividend',
                       'Pre-demographic dividend',
                       'Small states',
                       'South Asia',
                       'South Asia (IDA & IBRD)',
                       'Sub-Saharan Africa (IDA & IBRD countries)',
                       'Sub-Saharan Africa (excluding high income)',
                       'Upper middle income',
                       'World'
                       ]

# Drop the specified rows
df_population = df_population.filter(~df_population['country'].isin(countries_to_remove))

# Remove rows with missing values in the 'country' column
df_population = df_population.dropna(subset=['country'])

### $\textbf{Tax - Processamento}$

In [11]:
# Rename the column
df_tax = df_tax.withColumnRenamed("Country Name","country")
columns_to_drop = [str(year) for year in range(1960, 2000)]
df_tax = df_tax.drop("Country Code","Indicator Name","Indicator Code",*columns_to_drop,"_c67")

# Assuming df_tax is properly defined DataFrame
df_tax = df_tax.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("tax")) 
    for year in range(2000, 2023)
])).alias("data")).selectExpr("country", "data.year", "data.tax")

df_tax = df_tax.orderBy("country")

# Replace null values in 'total_cases' column with 0
df_tax = df_tax.withColumn("tax", when(df_tax["tax"].isNull(), 0).otherwise(df_tax["tax"]))

# Cast columns to their desired types
df_tax = df_tax.withColumn("country", col("country").cast("string"))
df_tax = df_tax.withColumn("year", col("year").cast("int"))
df_tax = df_tax.withColumn("tax", col("tax").cast("double"))

# Filter data for years greater than 2010 and lower than 2024
df_tax = df_tax.filter(df_tax["year"] > 2010)
df_tax = df_tax.filter(df_tax["year"] < 2024)

# Order by country and then by year
df_tax = df_tax.orderBy("country", "year")

replacements = {
    "China, People's Republic of": "China",
    "Gambia, The": "Gambia",
    "Micronesia, Fed. States of": "Micronesia",
    "South Sudan, Republic of": "South Sudan",
    "Taiwan Province of China": "Taiwan",
    "Türkiye, Republic of": "Türkiye",
    "Korea, Republic of": "South Korea",
    "Congo": "Democratic Republic of the Congo",
    "Caribbean small states": "Caribbean"
}

# Rename columns based on replacements dictionary
for old_value, new_value in replacements.items():
    df_tax = df_tax.withColumnRenamed(old_value, new_value)

# Columns to drop
countries_to_remove = ['Africa Eastern and Southern', 
                       'Africa Western and Central', 
                       'Central Europe and the Baltics', 
                       'Early-demographic dividend', 
                       'East Asia & Pacific', 
                       'East Asia & Pacific (IDA & IBRD countries)', 
                       'East Asia & Pacific (excluding high income)', 
                       'Euro area',
                       'Europe & Central Asia',
                       'Europe & Central Asia (IDA & IBRD countries)',
                       'Europe & Central Asia (excluding high income)',
                       'European Union',
                       'Fragile and conflict affected situations',
                       'Heavily indebted poor countries (HIPC)',
                       'High income',
                       'IBRD only',
                       'IDA & IBRD total',
                       'IDA blend',
                       'IDA only',
                       'IDA total',
                       'Latin America & the Caribbean (IDA & IBRD countries)',
                       'Latin America & Caribbean (excluding high income)',
                       'Least developed countries: UN classification',
                       'Low & middle income',
                       'Low income',
                       'Lower middle income',
                       'Middle East & North Africa',
                       'Middle East & North Africa (IDA & IBRD countries)',
                       'Middle East & North Africa (excluding high income)',
                       'Middle income',
                       'Not classified',
                       'OECD members',
                       'Other small states',
                       'Pacific island small states'
                       'Post-demographic dividend',
                       'Pre-demographic dividend',
                       'Small states',
                       'South Asia',
                       'South Asia (IDA & IBRD)',
                       'Sub-Saharan Africa (IDA & IBRD countries)',
                       'Sub-Saharan Africa (excluding high income)',
                       'Upper middle income',
                       'World'
                       ]

# Drop the specified rows
df_tax = df_tax.filter(~df_tax['country'].isin(countries_to_remove))

# Remove rows with missing values in the 'country' column
df_tax = df_tax.dropna(subset=['country'])

### $\textbf{Unemployment - Processamento}$

In [12]:
# Remove columns Country Code and Indicator Code
df_unemployment.drop("Country Code", "Indicator Code", "Indicator Name")

# Rename the column
df_unemployment = df_unemployment.withColumnRenamed("Country Name","country")

# Assuming df_unemployment is properly defined DataFrame
df_unemployment = df_unemployment.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("unemployment")) 
    for year in range(2000, 2023)
])).alias("data")).selectExpr("country", "data.year", "data.unemployment")

df_unemployment = df_unemployment.orderBy("country")

# Cast columns to their desired types
df_unemployment = df_unemployment.withColumn("country", col("country").cast("string"))
df_unemployment = df_unemployment.withColumn("year", col("year").cast("int"))
df_unemployment = df_unemployment.withColumn("unemployment", col("unemployment").cast("double"))

# Filter data for years greater than 2010 and lower than 2024
df_unemployment = df_unemployment.filter(df_unemployment["year"] > 2010)
df_unemployment = df_unemployment.filter(df_unemployment["year"] < 2024)

# Order by country and then by year
df_unemployment = df_unemployment.orderBy("country", "year")

replacements = {
    "China, People's Republic of": "China",
    "Gambia, The": "Gambia",
    "Micronesia, Fed. States of": "Micronesia",
    "South Sudan, Republic of": "South Sudan",
    "Taiwan Province of China": "Taiwan",
    "Türkiye, Republic of": "Türkiye",
    "Caribbean small states": "Caribbean",
    "Hong Kong SAR": "Hong Kong",
    "Congo": "Democratic Republic of the Congo",
    "Viet Nam": "Vietnam"
}

# Rename columns based on replacements dictionary
for old_value, new_value in replacements.items():
    df_unemployment = df_unemployment.withColumnRenamed(old_value, new_value)

# Columns to drop
countries_to_remove = ['Africa Eastern and Southern', 
                       'Africa Western and Central', 
                       'Central Europe and the Baltics', 
                       'Early-demographic dividend', 
                       'East Asia & Pacific', 
                       'East Asia & Pacific (IDA & IBRD countries)', 
                       'East Asia & Pacific (excluding high income)', 
                       'Euro area',
                       'Europe & Central Asia',
                       'Europe & Central Asia (IDA & IBRD countries)',
                       'Europe & Central Asia (excluding high income)',
                       'European Union',
                       'Fragile and conflict affected situations',
                       'Heavily indebted poor countries (HIPC)',
                       'High income',
                       'IBRD only',
                       'IDA & IBRD total',
                       'IDA blend',
                       'IDA only',
                       'IDA total',
                       'Latin America & the Caribbean (IDA & IBRD countries)',
                       'Latin America & Caribbean (excluding high income)',
                       'Least developed countries: UN classification',
                       'Low & middle income',
                       'Low income',
                       'Lower middle income',
                       'Middle East & North Africa',
                       'Middle East & North Africa (IDA & IBRD countries)',
                       'Middle East & North Africa (excluding high income)',
                       'Middle income',
                       'Not classified',
                       'OECD members',
                       'Other small states',
                       'Pacific island small states'
                       'Post-demographic dividend',
                       'Pre-demographic dividend',
                       'Small states',
                       'South Asia',
                       'South Asia (IDA & IBRD)',
                       'Sub-Saharan Africa (IDA & IBRD countries)',
                       'Sub-Saharan Africa (excluding high income)',
                       'Upper middle income',
                       'World',
                       'ther small states'
                       ]

# Drop the specified rows
df_unemployment = df_unemployment.filter(~df_unemployment['country'].isin(countries_to_remove))

# Remove rows with missing values in the 'country' column
df_unemployment = df_unemployment.dropna(subset=['country'])

### $\textbf{Merge Files}$

In [13]:
dfs = [df_covid,df_gdp,df_inflation,df_migration,df_population,df_tax,df_unemployment]

# Merge or outer join the DataFrames
merged_df = dfs[0]

for df in dfs[1:]:
    merged_df = merged_df.join(df, on=["country","year"], how="outer")

replacements = {
    "Korea": "North Korea",
    "Pacific island small states": "Pacific Islands"
}

# Rename columns based on replacements dictionary
for old_value, new_value in replacements.items():
    merged_df = merged_df.withColumnRenamed(old_value, new_value)

# Drop the specified columns
columns_to_drop = ['t. Lucia',
                   'ali', 
                   'alau',
                   'Anguilla',
                   'Bonaire Sint Eustatius and Saba',
                   'Cook Islands',
                   'American Samoa',
                   'Arab World',
                   'European Union',
                   'Faeroe Islands',
                   'Falkland Islands',
                   'Faroe Islands',
                   'French Guiana',
                   'Guadeloupe',
                   'Guernsey',
                   'High income',
                   'Isle of Man',
                   'Jersey',
                   'Kyrgyzstan',
                   'Late-demographic dividend',
                   'Latin America & Caribbean',
                   'Low income',
                   'Macao',              
                   'Martinique',
                   'Mayotte',
                   'Montserrat',
                   'Niue',
                   'North America',
                   'North Korea',
                   'Northern Cyprus',
                   'Northern Ireland',
                   'Oceania',
                   'Western Sahara',
                   'Wales',
                   'Wallis and Futuna',
                   'Vatican',
                   'Tokelau',
                   'Sub-Saharan Africa',
                   'Sub-Saharan Africa (Region) ',
                   'St. Lucia',
                   'St. Martin',
                   'Sint Maarten',
                   'Sint Maarten (Dutch part)',
                   'Scotland',
                   'Saint Martin',
                   'Saint Lucia',
                   'Saint Helena',
                   'Reunion',
                   'Post-demographic dividend',
                   'Pitcairn',
                   'Palestine',
                   'England',
                   'Saint Pierre and Miquelon',
                   'Saint Barthelemy',
                   'South America',
                   'St. Martin (French part)'
                   ]

# Drop the specified rows
merged_df = merged_df.filter(~df['country'].isin(columns_to_drop))

# Rename "no data" to "nan"
merged_df = merged_df.withColumn("country", when(df["country"] == "no data", "nan").otherwise(df["country"]))
merged_df = merged_df.withColumn("covid_cases", when(col("covid_cases").isNull(), 0).otherwise(col("covid_cases")))

# Sort the merged data by country and then by year
merged_df = merged_df.orderBy(col('country'), col('year'))

#Storing this dataframe in parquet
merged_df.write.mode("overwrite").parquet("FinalAllInOneParquet")
spark.read.parquet("FinalAllInOneParquet").show()
spark.stop()

Py4JJavaError: An error occurred while calling o1916.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.

In [ ]:
fim = time.time()
final = fim - inicio
print(final)

14.308474779129028
